<a href="https://colab.research.google.com/github/rosenanna/Master_Thesis/blob/main/image_processing/border_removal_of_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -U albumentations
!pip install opencv-python-headless==4.1.2.30

In [ ]:
import cv2
import albumentations as A
import numpy as np
from matplotlib import pyplot as plt
from google.colab.patches import cv2_imshow
import random
import os

In [ ]:
### Specify the number of images
for m in range(692, 1344+1):
    
    # image_path = '/content/drive/MyDrive/thesis/ELLA_Eivor1.0_DERM_dataset.zip (Unzipped Files)/test/smallTest'
    image_path = '/content/drive/MyDrive/ELLA_Eivor1.0_DERM_dataset.zip (Unzipped Files)/everything'
    path_in = os.path.join(image_path, "{:04d}".format(m)+ '_derm_01.png') 
    img = cv2.imread(path_in)
    if os.path.isfile(path_in) == False:
      continue
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    _,thresh = cv2.threshold(gray,1,255,cv2.THRESH_BINARY)
    # cv2_imshow(gray)

    threshold = 80
    shadow_threshold = 0.025*gray_cropped_image.shape[0]

    histo = np.histogram(gray,100,  [30,255])
    for i in range(histo[0].shape[0]-5, 0, -1):
      if histo[0][i-1] < np.mean([histo[0][i], histo[0][i+1], histo[0][i+2], histo[0][i+3], histo[0][i+4]]):
        threshold_skin = histo[1][i-1]
        break;
    threshold_skin = threshold_skin - 30

    # Finding where the pixels are above the threshold
    for j in range(gray.shape[1]):
      find = False
      for i in range(gray.shape[0]):
        if gray[i][j] >= threshold:
          y_init = j
          find = True
          break
      if find:
        break
    # Finding the cordinates from all 4 sides in the picture
    for j in range(gray.shape[1]-1,0,-1):
      find = False
      for i in range(gray.shape[0]-1,0,-1):
        if gray[i][j] >= threshold:
          y_fin = j
          find = True
          break
      if find:
        break

    for i in range(gray.shape[1]):
      find = False
      for j in range(gray.shape[0]):
        if gray[i][j] >= threshold:
          x_init = i
          find = True
          break
      if find:
        break

    for i in range(gray.shape[1]-1,0,-1):
      find = False
      for j in range(gray.shape[0]-1,0,-1):
        if gray[i][j] >= threshold:
          x_fin = i
          find = True
          break
      if find:
        break

    # crop_dims = np.array([x_init, y_init, x_fin, y_fin])
    # Cropping the image so that only what's above the threshold is included
    # for both the original and gray image
    cropped_image = img[y_init:y_fin, x_init:x_fin]
    gray_cropped_image = gray[y_init:y_fin, x_init:x_fin]
    good_pixels = []
    bad_pixels = []
    for j in range(gray_cropped_image.shape[1]):
      for i in range(gray_cropped_image.shape[0]):
        if gray_cropped_image[i][j] < 30:
          bad_pixels.append(np.array([i,j]))
        elif gray_cropped_image[i][j] > threshold_skin:
          good_pixels.append([i,j])
    good_pixels = np.array(good_pixels)
    bad_pixels = np.array(bad_pixels)
    # I'm a little cinfused as to what happens here?
    bad_list = np.random.randint(bad_pixels.shape[0], size=bad_pixels.shape[0])
    good_list = np.random.randint(good_pixels.shape[0], size=good_pixels.shape[0])

    bad_pixel_list = bad_pixels[bad_list]
    good_pixel_list = good_pixels[good_list]
    # cropped_image[bad_pixel_list] = cropped_image[good_pixel_list]
    
    temp = 0
    #Replacing "bad" pixels with good ones
    # 2nd quadrant
    for i in range(int(np.floor(gray_cropped_image.shape[0]/2))):
        for j in range(int(np.floor(gray_cropped_image.shape[1]/2))):
            
            if gray_cropped_image[i,j] < threshold or gray_cropped_image[i,j] < gray_cropped_image[i+1,j+1]:
                rand = np.random.randint(0, good_pixel_list.shape[0])
                cropped_image[i,j,:] = cropped_image[good_pixel_list[rand][0], good_pixel_list[rand][1], :] 
            elif temp > shadow_threshold:
                temp = 0
                break
            else :
                temp = temp +1

            

    # 1st quadrant
    for i in range(int(np.floor(gray_cropped_image.shape[0]/2))):
        for j in range(gray_cropped_image.shape[1]-3, int(np.floor(gray_cropped_image.shape[1]/2)-3), -1):
            # if gray_cropped_image[i,j] > 2*threshold:
            #     break
            if gray_cropped_image[i,j] < threshold or gray_cropped_image[i,j] < gray_cropped_image[i+1,j-1]:
                rand = np.random.randint(0, good_pixel_list.shape[0])
                cropped_image[i,j,:] = cropped_image[good_pixel_list[rand][0], good_pixel_list[rand][1], :] 
            elif temp > shadow_threshold:
                temp = 0
                break
            else :
                temp = temp +1

    # 3rd quadrant
    for i in range(gray_cropped_image.shape[0]-2, int(np.floor(gray_cropped_image.shape[0]/2)-3), -1):
        for j in range(int(np.floor(gray_cropped_image.shape[1]/2))):
            # if gray_cropped_image[i,j] > 2*threshold:
            #     break
            if gray_cropped_image[i,j] < threshold or gray_cropped_image[i,j] < gray_cropped_image[i-1,j+1]:
                rand = np.random.randint(0, good_pixel_list.shape[0])
                cropped_image[i,j,:] = cropped_image[good_pixel_list[rand][0], good_pixel_list[rand][1], :] 
            elif temp > shadow_threshold:
                temp = 0
                break
            else :
                temp = temp +1

    # 4th quadrant
    for i in range(gray_cropped_image.shape[0]-2, int(np.floor(gray_cropped_image.shape[0]/2)-3), -1):
        for j in range(gray_cropped_image.shape[1]-2, int(np.floor(gray_cropped_image.shape[1]/2)-3), -1):
            # if gray_cropped_image[i,j] > 2*threshold:
            #     break
            if gray_cropped_image[i,j] < threshold or gray_cropped_image[i,j] < gray_cropped_image[i-1,j-1]:
                rand = np.random.randint(0, good_pixel_list.shape[0])
                cropped_image[i,j,:] = cropped_image[good_pixel_list[rand][0], good_pixel_list[rand][1], :] 
            elif temp > shadow_threshold:
                temp = 0
                break
            else :
                temp = temp +1



    # path_out = os.path.join('/content/drive/MyDrive/pre_processed_images(different sizes)/', "{:04d}".format(m)+ '_derm_01.jpeg')
    path_out = os.path.join('/content/drive/MyDrive/ELLA_Eivor1.0_DERM_dataset.zip (Unzipped Files)/preprocessed/', "{:04d}".format(m)+ '_derm_01.jpeg')
    
    cv2.imwrite(path_out, cropped_image)